In [39]:
import pandas as pd

In [40]:
# read dataset
with open('../data/Zip_Zhvi_SingleFamilyResidence.csv', 'rb') as f:
    df = pd.read_csv(f).fillna(0)

# convert non-timeseries
df[['City', 'State', 'Metro', 'CountyName']] = df[['City', 'State', 'Metro', 'CountyName']].astype(str)
df[['RegionID', 'RegionName', 'SizeRank']] = df[['RegionID', 'RegionName', 'SizeRank']].astype(int)

In [41]:
# timeseries data
start_index = df.columns.get_loc('1996-04') + 1
date_columns = df.iloc[:,start_index:len(df.columns)].columns.tolist()

# ensure integer timeseries
df[date_columns] = df[date_columns].astype(int)